In [1]:
from common import *
from search.imports import *
from search.file import *
import FDF as _FDF, FDS as _FDS, FDP as _FDP

## Model Manager

In [ ]:
class ModelManager():
    features_dir = '.features'
    samples_dir = '.samples'
    models_dir = 'models'
    
    def __init__(self, directory, f_specs=None, s_specs=None, p_specs=None):
        if f_specs is None:
            f_specs = _FDF.FDF
        if s_specs is None:
            s_specs = _FDS.FDS
        if p_specs is None:
            p_specs = _FDP.FDP
        self.p_specs = p_specs
        self.dir = directory if isinstance(directory, Path) else Path(directory)
        self.dir.mkdir(exist_ok=True) # no parents because this is user facing and enabling parents could get gnarly
        self.F = IndexDataStore.from_specs(f_specs, dir=self.dir/self.features_dir)
        self.S = IndexDataStore.from_specs(s_specs, dir=self.dir/self.samples_dir)
        (self.dir/self.models_dir).mkdir(exist_ok=True)
        self.load()
        
    def load(self):
        self.pms = {}
        for fi, si in self.iterIndices():
            self.make_params_manager(i=(fi, si))
        
    def iterIndices(self):
        for fdir in (self.dir/self.models_dir).iterdir():
            try:
                fi = int(fdir.name)
            except ValueError:
                continue
            assert fdir.is_dir(), "folder in models folder whose name is just a number must be a folder"
            assert fi < self.F.nextIndex, "features folder found with greater index than what has been labelled"
            for sdir in fdir.iterdir():
                try:
                    si = int(sdir.name)
                except ValueError:
                    continue
                assert sdir.is_dir(), "folder in models folder 1 layer down whose name is just a number must be a folder"
                assert si < self.S.nextIndex, "samples folder found with greater index than what has been labelled"
                yield (fi, si)
        
    def make_params_manager(self, features=None, samples=None, i=None):
        assert (
            not ((features is None) ^ (samples is None)) and
            ((features is not None and samples is not None) ^ (i is not None)),
            "invalid arguments to `make_param_manager`"
        )
        if i is None:
            i = self.F.i(features), self.S.i(samples)
        fi, si = i
        assert 0 <= fi < self.F.nextIndex and 0 <= si < self.S.nextIndex, "bad arguments to `make_params_manager`"
        pm = IndexDataStore.from_specs(self.p_specs, dir=self.dir/self.models_dir/str(fi)/str(si))
        self.pms[fi, si] = pm
        return pm
    
    def i(features, samples): # purely convenience public interface
        return self.F.i(features), self.S.i(samples)
        
    def PM(self, features, samples, get_i=False):
        fi, si = self.F.i(features), self.S.i(samples)
        if get_i:
            return self.iPM(fi, si), (fi, si)
        else:
            return self.iPM(fi, si)
        
    def iPM(self, fi, si):
        if (fi, si) not in self.pms:
            self.make_params_manager(i=(fi, si))
        return self.pms[fi, si]